In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [1]:
#import the gym required files
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

#import helpers
import numpy as np
import random
import os

#import stable baselines models
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

C:\Users\alama\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\alama\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 2. Types of Spaces

In [2]:
Discrete(3)

Discrete(3)

In [4]:
Box(0,1,shape=(3,3)).sample()

array([[2.5651822e-04, 9.2248917e-01, 4.5637864e-01],
       [8.6772573e-01, 1.3512383e-02, 4.1818568e-01],
       [5.0744329e-02, 2.2877219e-01, 4.8027539e-01]], dtype=float32)

In [5]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 70, 163,  96],
       [169,  81, 191],
       [178,  53, 230]])

In [6]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([55.94329], dtype=float32))

In [7]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([26.325897], dtype=float32))])

In [8]:
MultiBinary(4).sample()

array([0, 0, 1, 1], dtype=int8)

In [13]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 0], dtype=int64)

# 3. Building an Environment

In [14]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=0, high = 100, shape = (1,))
        # Set start temp
        #    self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [15]:
env=ShowerEnv()

In [16]:
env.observation_space.sample()

array([27.969866], dtype=float32)

In [17]:
env.reset()

array([38.])

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(env, warn=True)

# 4. Test Environment

In [18]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-4
Episode:2 Score:-18
Episode:3 Score:-46
Episode:4 Score:-54
Episode:5 Score:-12


In [ ]:
env.close()

# 5. Train Model

In [19]:
log_path = os.path.join('Training', 'Logs')

In [20]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\alama\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [22]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -19.4    |
| time/              |          |
|    fps             | 1855     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -18.4       |
| time/                   |             |
|    fps                  | 1191        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007947322 |
|    clip_fraction        | 0.065       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.634      |
|    explained_variance   | 0.0543      |

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -8.28        |
| time/                   |              |
|    fps                  | 843          |
|    iterations           | 11           |
|    time_elapsed         | 26           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0024956316 |
|    clip_fraction        | 0.0373       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.403       |
|    explained_variance   | -0.0173      |
|    learning_rate        | 0.0003       |
|    loss                 | 52.2         |
|    n_updates            | 340          |
|    policy_gradient_loss | -0.00117     |
|    value_loss           | 110          |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 6.4          |
| time/                   |              |
|    fps                  | 811          |
|    iterations           | 20           |
|    time_elapsed         | 50           |
|    total_timesteps      | 40960        |
| train/                  |              |
|    approx_kl            | 0.0011109031 |
|    clip_fraction        | 0.018        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.316       |
|    explained_variance   | -0.00394     |
|    learning_rate        | 0.0003       |
|    loss                 | 68.6         |
|    n_updates            | 430          |
|    policy_gradient_loss | 0.000776     |
|    value_loss           | 114          |
------------------------------------------


# 6. Save Model

In [25]:
shower_path = os.path.join('Training','Saved Models','Shower_Model_PPO')

In [26]:
model.save(shower_path)

In [27]:
del model

In [29]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [31]:
evaluate_policy(model,env,n_eval_episodes=10,render = True)

(12.0, 58.787753826796276)